In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
import time
import pandas as pd
import requests
import os
import re

#저장 경로 설정
SAVE_DIR = "C:/Users/user080524/Desktop/project/PREMO/data/datas/1"
os.makedirs(SAVE_DIR, exist_ok=True)

#셀레니움 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(), options=chrome_options)


In [17]:
def crawl(season_url, team_index):
    print(f"[시즌 페이지 로딩] URL: {season_url}")
    driver.get(season_url)
    time.sleep(2)
    
    try:
        
        match = re.search(r'/(\d{4}-\d{4})/', season_url)
        if match:
            season_code = match.group(1)
        else:
            season_code = '2024-2025'
        id = "results" + season_code + "91_overall"
        
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, id))
            )
        if not table:
            print("팀 테이블 없음")
            return []
        
        rows = table.find_elements(By.CSS_SELECTOR, 'tbody > tr')
        if team_index >= len(rows):
            print(f"team_index {team_index} out of range")
            return []
    
        row = rows[team_index]    
        link = row.find_element(By.CSS_SELECTOR, 'td[data-stat="team"] a')
        team_name = link.text.strip()
        team_href = link.get_attribute('href')
        print(f"팀: {team_name}, 링크:{team_href}")
        
        driver.get(team_href)
        time.sleep(2)


        team_results=[]
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "matchlogs_for")))
            match_table = driver.find_element(By.ID, "matchlogs_for")
            if not match_table:
                print("match 테이블 없음")
                return []
            
            match_rows = match_table.find_elements(By.CSS_SELECTOR, 'tbody > tr')
            if not match_rows:
                print("match_rows 비어 있음")
                return []   
    
            for match_row in match_rows:
                try:
                    comp_value = match_row.find_element(By.CSS_SELECTOR, 'td[data-stat="comp"]').text.strip()
                    venue = match_row.find_element(By.CSS_SELECTOR, 'td[data-stat="venue"]').text.strip()
                    if comp_value != "Premier League" or venue != "Home":
                        continue
                    
                    match = match_row.find_element(By.CSS_SELECTOR, 'th[data-stat="date"]')
                    match_date = match.get_attribute("csk") or "unknown_date"
                    
                    try:
                        match_link = match.find_element(By.TAG_NAME, 'a').get_attribute('href')
                    except:
                        match_link = None
                    
                    if match_link: 
                        team_results.append({
                            "team":team_name,
                            "date":match_date,
                            "link":match_link
                        })
                        print(match_date)
                        print(match_link)
                except Exception as e:
                    print("match row 처리 중 오류:", e)
                    continue
                
            print(len(team_results))
                
        except Exception as e:
            print("match 테이블 오류:", e)
        
        return team_results
    
    except Exception as e:
        print("시즌 오류 발생:", e)
        return []

In [ ]:
#시즌별 링크
season_urls = []

for start_year in range(2021, 2022):
    end_year = start_year+1
    season = f"{start_year}-{end_year}"
    url = f"https://fbref.com/en/comps/9/{season}/{season}-Premier-League-Stats"
    season_urls.append(url)
season_urls.append("https://fbref.com/en/comps/9/Premier-League-Stats")


for link in tqdm(season_urls, desc="Season URLs"):
    season_match = re.search(r'(\d{4}-\d{4})', link)
    season = season_match.group(1) if season_match else "unknown_season"
    
    for i in tqdm(range(20), desc=f"Teams in {season}"):
        data = crawl(link, i)
        print(len(data))
        
        for match in tqdm(data, desc=f"Matches for Team{i}", leave=False):
            match_url = match["link"]
            driver.get(match_url)
            
            time.sleep(2)
            
            save_folder = os.path.join(SAVE_DIR, season, match["team"], match["date"])
            os.makedirs(save_folder, exist_ok=True)
            
            #players data summary + gk(처음 4개 테이블)
            player_tables = driver.find_elements(By.CLASS_NAME, "stats_table.sortable.now_sortable.sticky_table.eq1.re1.le1")
            if not player_tables or len(player_tables) < 4:
                print("선수 테이블 개수 오류")
                continue
            
            summary_gk_labels = ["home_palyers_summary", "home_gk", "away_players_summary", "away_gk"]
            
            for idx, table in enumerate(player_tables[:4]):
                try:
                    thead = table.find_element(By.CSS_SELECTOR, "thead")
                    tbody = table.find_element(By.CSS_SELECTOR, "tbody")
                except Exception as e:
                    print("thead 또는 tbody 없음")
                    continue
                
                players_datas=[]
                row_elements = tbody.find_elements(By.TAG_NAME, "tr")
                
                for row_elem in row_elements:
                    row_data = {}
                    try:
                        th_elem = row_elem.find_element(By.TAG_NAME, "th")
                        stat_name = th_elem.get_attribute("data-stat")
                        value = th_elem.text.strip()
                        row_data[stat_name] = value
                    except Exception as e:
                        print("헤더 추출 오류", e)
                        pass
                    
                    td_elements = row_elem.find_elements(By.TAG_NAME, "td")
                    for td in td_elements:
                        stat_name = td.get_attribute("data-stat")
                        value = td.text.strip()
                        row_data[stat_name] = value
                    
                    if row_data:
                        players_datas.append(row_data)
                    
                df = pd.DataFrame(players_datas)
                
                match_date = match["date"]
                label = summary_gk_labels[idx]
                file_name = f"{match_date}_{label}.csv"
                save_path = os.path.join(save_folder, file_name)
                df.to_csv(save_path, index=False, encoding="utf-8-sig")
                print(f"[players stats summary 저장 완료] {save_path}")
            
            
            
            #team stats 추출
            try:
                team_stats = driver.find_element(By.ID, "team_stats")
                tbody = team_stats.find_element(By.TAG_NAME, "tbody")
                trs = tbody.find_elements(By.TAG_NAME, "tr")
                
                target_indices = [2,4,6,8]
                stat_names = ["Possession", "Passing Accuracy", "Shots on Target", "Saves"]
                
                stats = {}
                for i, idx in enumerate(target_indices):
                    tr = trs[idx]
                    tds = tr.find_elements(By.TAG_NAME, "td")
                    home_val = float(tds[0].find_element(By.TAG_NAME, "strong").text.replace('%', ''))
                    away_val = float(tds[1].find_element(By.TAG_NAME, "strong").text.replace('%', ''))
                    stats[f"{stat_names[i]}_home"] = home_val
                    stats[f"{stat_names[i]}_away"] = away_val
            except Exception as e:
                print("team_stats 없음", e)
                stats = {}
            
            #team extra stats 추출
            try:
                container = driver.find_element(By.ID, "team_stats_extra")
                blocks = container.find_elements(By.XPATH, "./div")
                
                extra_stats = {}
                for block in blocks:
                    divs = block.find_elements(By.TAG_NAME, "div")
                    
                    i = 3
                    while i+2 < len(divs):
                        home_val = divs[i].text.strip()
                        stat_name = divs[i+1].text.strip()
                        away_val = divs[i+2].text.strip()
                        
                        if home_val.isdigit() and away_val.isdigit():
                            extra_stats[f"{stat_name}_home"] = int(home_val)
                            extra_stats[f"{stat_name}_away"] = int(away_val)
                        i+=3
            except Exception as e:
                print("team stats extra 없음", e)
                extra_stats = {}
                
            combined_stats = {**stats, **extra_stats}
            df = pd.DataFrame([combined_stats])
            match_date = match["date"]
            file_name = f"{match_date}_team_stats.csv"
            #save_folder = os.path.join(SAVE_DIR, match_date)
            save_path = os.path.join(save_folder, file_name)
            df.to_csv(save_path, index=False, encoding="utf-8-sig")
            print(f"[team_stats 저장 완료] {save_path}")
            
            
            
            
            
driver.quit()

Season URLs:   0%|          | 0/2 [00:00<?, ?it/s]

[시즌 페이지 로딩] URL: https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats
팀: Manchester City, 링크:https://fbref.com/en/squads/b8fd03ef/2021-2022/Manchester-City-Stats
20210821
https://fbref.com/en/matches/ab6db0d6/Manchester-City-Norwich-City-August-21-2021-Premier-League
20210828
https://fbref.com/en/matches/d4650aa2/Manchester-City-Arsenal-August-28-2021-Premier-League
20210918
https://fbref.com/en/matches/1576c578/Manchester-City-Southampton-September-18-2021-Premier-League
match row 처리 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":"td[data-stat="comp"]"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7535BCF45+75717]
	GetHandleVerifier [0x00007FF7535BCFA0+75808]
	(No symbol) [0x00007FF753388F9A]
	(No symbol) [0x00007FF7533DF4C6]
	(No symbol) [0x0

[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210821\20210821_home_palyers_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210821\20210821_home_gk.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210821\20210821_away_players_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210821\20210821_away_gk.csv


team_stats 없음 could not convert string to float: ''
[team_stats 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210821\20210821_team_stats.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210828\20210828_home_palyers_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210828\20210828_home_gk.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210828\20210828_away_players_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210828\20210828_away_gk.csv


team_stats 없음 could not convert string to float: ''
[team_stats 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210828\20210828_team_stats.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210918\20210918_home_palyers_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210918\20210918_home_gk.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210918\20210918_away_players_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\2021-2022\Manchester City\20210918\20210918_away_gk.csv


...

[team_stats 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250413\20250413_team_stats.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250427\20250427_home_palyers_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250427\20250427_home_gk.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250427\20250427_away_players_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250427\20250427_away_gk.csv


[team_stats 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250427\20250427_team_stats.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250511\20250511_home_palyers_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250511\20250511_home_gk.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250511\20250511_away_players_summary.csv
[players stats summary 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250511\20250511_away_gk.csv


[team_stats 저장 완료] C:/Users/user080524/Desktop/project/PREMO/data/datas/1\unknown_season\Liverpool\20250511\20250511_team_stats.csv
[시즌 페이지 로딩] URL: https://fbref.com/en/comps/9/Premier-League-Stats


Season URLs:  50%|█████     | 1/2 [1:07:56<1:07:56, 4076.91s/it]


KeyboardInterrupt: 